In [ ]:
# Refer to https://pythonawesome.com/overview-of-the-peaks-dectection-algorithms-available-in-python/  for finding peaks
#df = pd.read_excel('sample_data/20191129_A028300.xlsx')

In [1]:
from morning.pipeline.stream.cybos.stock.db.min_tick import MinTick
from morning.pipeline.chooser.cybos.db.kosdaq_all_chooser import KosdaqAllChooser
from morning.back_data.fetch_stock_data import get_day_minute_period_data
from scipy.signal import argrelextrema
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
from datetime import date
kac = KosdaqAllChooser()
kac.set_date(date(2019, 1, 2))
codes = kac.codes
len(codes)

LEN 12
FILTER A090410 1393
DB LEN 12
FILTER A090460 1393
DB LEN 12
FILTER A090470 1393
DB LEN 12
FILTER A090710 1393
DB LEN 12
FILTER A090740 1393
DB LEN 12
FILTER A090850 1393
DB LEN 12
FILTER A091120 1393
DB LEN 12
FILTER A091340 1393
DB LEN 12
FILTER A091440 1393
DB LEN 12
FILTER A091580 1393
DB LEN 12
FILTER A091590 1393
DB LEN 12
FILTER A091700 1393
DB LEN 12
FILTER A091970 1393
DB LEN 12
FILTER A091990 1393
DB LEN 12
FILTER A092040 1393
DB LEN 12
FILTER A092070 1393
DB LEN 12
FILTER A092130 1393
DB LEN 12
FILTER A092300 1393
DB LEN 12
FILTER A092460 1393
DB LEN 12
FILTER A092600 1393
DB LEN 12
FILTER A092730 1393
DB LEN 12
FILTER A092870 1393
DB LEN 12
FILTER A093190 1393
DB LEN 12
FILTER A093320 1393
DB LEN 12
FILTER A093380 1393
DB LEN 12
FILTER A093520 1393
DB LEN 12
FILTER A093640 1393
DB LEN 12
FILTER A093920 1393
DB LEN 12
FILTER A094170 1393
DB LEN 12
FILTER A094190 1393
DB LEN 12
FILTER A094360 1393
DB LEN 12
FILTER A094480 1393
DB LEN 12
FILTER A094820 1393
DB LEN 12
FIL

714

In [40]:
from datetime import datetime, date
from_date = date(2019, 1, 2)
code_index = 0

In [64]:
code = codes[code_index]
data = get_day_minute_period_data(code, from_date, from_date)
for d in data:
    d['date'] = datetime(int(d['0'] / 10000), int(d['0'] % 10000 / 100), int(d['0'] % 100), int(d['1'] / 100), int(d['1']% 100))
df = pd.DataFrame(data)

code_index += 1

df['avg'] = df['5'] # close_price
minima = argrelextrema(np.nan_to_num(df['avg'].to_numpy()), np.less, order=10)
maxima = argrelextrema(np.nan_to_num(df['avg'].to_numpy()), np.greater, order=10)
df['minima'] = False
df['maxima'] = False
df['minima'].iloc[minima] = True
df['maxima'].iloc[maxima] = True
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=df['date'], y=df['avg'], mode='lines'))
fig.add_trace(go.Scatter(x=df['date'], y=df['9'], name='culmulate buy', line=dict(color='red')), row=2, col=1)
fig.add_trace(go.Scatter(x=df['date'], y=df['8'], name='culmulate sell', line=dict(color='blue')), row=2, col=1)
d = df[df['minima'] == True]['date']
d2 = df[df['maxima'] == True]['date']
shapes = []
shapes_maxima = []
for date in d:
    shapes.append(dict(x0=date, x1=date, y0=0, y1=1, xref='x', yref='paper', line=dict(color='magenta')))

for date in d2:
    shapes_maxima.append(dict(x0=date, x1=date, y0=0, y1=1, xref='x', yref='paper', line=dict(color='black')))
shapes.extend(shapes_maxima)
fig.update_layout(yaxis_tickformat='d', shapes=shapes)


DB LEN 320
